# Stock data analysis

In [1]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

# Connecting to OpenAI

In [2]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [3]:
openai = OpenAI()

In [4]:
from urllib.parse import urlencode
import datetime

headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

In [5]:
class YahooFinanceWebsite:
    def __init__(self, stock_symbol):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.stock_symbol = stock_symbol.upper()

    def __build_url(self, params):
        base_url = f"https://finance.yahoo.com/quote/{self.stock_symbol}/history/"
        query_string = urlencode(params)
        return f"{base_url}?{query_string}"

    def get_stock_data(self):
        datetime_now = datetime.datetime.now()
        datetime_year_ago = datetime_now - datetime.timedelta(days=365)
        params = {"frequency": "1wk", "period1": datetime_year_ago.timestamp(), "period2": datetime_now.timestamp()}
        url = self.__build_url(params)
        response = requests.get(url, headers=headers)

        soup = BeautifulSoup(response.content, 'html.parser')
        
        title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()

        html_table_data = soup.find("table")

        return title, html_table_data

In [6]:
def build_stock_analysis_prompt(stock_symbol, title, stock_table_data):
    sys_prompt = r"""You are an assistant that analyzes the contents of HTML formated table that contains data on a specific stock.
    The HTML table contains the date, open price, close price, low and highs aggregated for every week over one year timeframe.
    Ignoring text, tags or html attributes that might be navigation related. 
    Respond in Markdown format"""
    
    user_prompt = f"The data provided below in the HTML table format for {stock_symbol} from the Yahoo Finances.\
    Make the explaination easy enough for a newbie to understand. \
    Analyze and Summarize the trends on this stock:\n{stock_table_data}\n\n\
    Also, calculate the total returns in percentage one could have expected over this period."
    
    return [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": user_prompt}
    ]

In [7]:
def analyze_stock_trends(stock_symbol):
    stock_data_page = YahooFinanceWebsite(stock_symbol)
    title, stock_table_data = stock_data_page.get_stock_data()
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = build_stock_analysis_prompt(stock_symbol, title, stock_table_data)
    )
    return response.choices[0].message.content

def display_analysis(stock_symbol):
    display(Markdown(analyze_stock_trends(stock_symbol)))

In [8]:
display_analysis("GOOG")

## Stock Performance Analysis for GOOG

The following is an analysis based on the provided data from the weekly performance of Google (GOOG) stock over a year. Here’s a breakdown of the key trends and observations:

### Key Data Metrics

- **Date Range**: The last recorded week was on March 21, 2025.
- **Opening Price**: The price at which the stock opened at the start of the period was generally high, around $163.32.
- **Closing Price**: The closing price also displayed variability with a notable peak reaching as high as $206.25 at some points.
- **High and Low Prices**: Price variation indicates several peaks and troughs, with highs occasionally exceeding $200, while lows dipped to around $148.

### Summary of Trends

1. **Overall Upward Trend**:
   - Starting from around $150 on March 25, 2024, the stock reached approximately $166 by March 21, 2025. This suggests a gradual increase in the stock price over this period despite fluctuations.
   
2. **Price Volatility**:
   - The stock experienced several fluctuations where the high price during the week often exceeded $200, signaling periods of strong market interest.
   - The lows reflected corrections; for instance, the lowest weekly closing was around $148, suggesting some weeks where buying pressure was lower.

3. **Volume Trends**:
   - There was a considerable trading volume in many weeks, especially during price fluctuations. The weekly volumes often surpassed tens of millions, showing active trading.

4. **Dividends**:
   - There were multiple dividends noted (e.g., $0.20 per share weeks), reflecting a strategy to share profits with shareholders.

### Returns Calculation

To calculate the total returns based on closing prices:

- **Starting Price (Closing Price on March 25, 2024)**: $152.26
- **Ending Price (Closing Price on March 21, 2025)**: $166.25

#### Calculation:

\[
\text{Total Return} = \left(\frac{\text{Ending Price} - \text{Starting Price}}{\text{Starting Price}}\right) \times 100
\]

Substituting the values in:

\[
\text{Total Return} = \left(\frac{166.25 - 152.26}{152.26}\right) \times 100 \approx \left(\frac{13.99}{152.26}\right) \times 100 \approx 9.17\%
\]

### Conclusion

- **Total Returns**: An investor in GOOG could have expected approximately **9.17%** total returns over the analyzed period.
- The continued interest in Google stock, indicated by consistent trading volumes and dividends, along with the upward trajectory, suggests that it remains a strong candidate for investors looking to hold positions in tech stocks.

This summary reflects the trends and performance of Google's stock based on the provided data, offering insights into trading behaviors and potential investment opportunities.

In [ ]:
display_analysis("PFE")

In [ ]:
display_analysis("AAPL")